In [1]:
import requests
import urllib
import urllib2
import re
import json
import pandas as pd
import xml.etree.ElementTree as ET

from collections import defaultdict
from bs4 import BeautifulSoup
from scrapy import Selector
from scrapy.selector import HtmlXPathSelector
from weather import Weather

In [2]:
search_text = "Chennai"

<h4>Yahoo Weather Information</h4>

In [3]:
try:
    weather = Weather()
    location = weather.lookup_by_location(search_text)
    condition = location.condition()
    yahoo_data_now = ("Yahoo", int((int(condition['temp'])-32)*float(0.5556)), "", "", "")

    yahoo_url = "https://query.yahooapis.com/v1/public/yql?"
    yql_query = "select * from weather.forecast where woeid in (select woeid from geo.places(1) where text='{}')".format(search_text)
    yql_url = yahoo_url + urllib.urlencode({'q':yql_query}) + "&format=json"
    result = urllib2.urlopen(yql_url).read()
    data = json.loads(result)

    week_data = data['query']['results']["channel"]["item"]["forecast"]
    day_1 = int((int(week_data[1]['high']) - 32) * 0.5556)
    day_2 = int((int(week_data[2]['high']) - 32) * 0.5556)
    day_3 = int((int(week_data[3]['high']) - 32) * 0.5556)
    day_4 = int((int(week_data[4]['high']) - 32) * 0.5556)
    day_5 = int((int(week_data[5]['high']) - 32) * 0.5556)
    yahoo_data_now = ("Yahoo", int((int(condition['temp'])-32)*float(0.5556)), day_1, day_2, day_3, day_4, day_5)
except:
    yahoo_data_now = (
        "Yahoo",
        "NA",
        "NA",
        "NA",
        "NA",
        "NA",
        "NA",
    )
print yahoo_data_now

('Yahoo', 29, 28, 29, 30, 31, 31)


<h4>The Weather Network</h4>

In [4]:
base_url = "https://www.theweathernetwork.com/in/api/location/search?searchText={}".format(search_text)
req = requests.get(base_url)

data = req.json()
data_url = []
if data:
    for i in data:
        if i["name"] == search_text.title() and i["country"] == "India":
            each_url = "https://www.theweathernetwork.com{}".format(i["url"])            
            data_url.append({"each_url": each_url, "code": i["code"]})

try:            
    if data_url:
    
        req_url = data_url[0]
        current_weather_url = "https://www.theweathernetwork.com/in/api/savedlocation/index/?placecodes={}".format(req_url["code"])
        current_weather_req = requests.get(current_weather_url).json()
        current_day_temp = current_weather_req[0]["temperature"]
    
        resp = urllib2.urlopen(req_url["each_url"]).read()
        soup = BeautifulSoup(resp, "lxml")
    
        current_data_day_1 = soup.find_all("div", attrs={"class": "day_1"})
        body = """{}""".format(current_data_day_1[0])
        selection_day_1 = Selector(text=body).xpath("//div").extract()
        temp_day_1 = re.findall("\d+", selection_day_1[5])[0]

        current_data_day_2 = soup.find_all("div", attrs={"class": "day_2"})
        body = """{}""".format(current_data_day_2[0])
        selection_day_2 = Selector(text=body).xpath("//div").extract()
        temp_day_2 = re.findall("\d+", selection_day_2[5])[0]

        current_data_day_3 = soup.find_all("div", attrs={"class": "day_3"})
        body = """{}""".format(current_data_day_3[0])
        selection_day_3 = Selector(text=body).xpath("//div").extract()
        temp_day_3 = re.findall("\d+", selection_day_3[5])[0]

        current_data_day_4 = soup.find_all("div", attrs={"class": "day_4"})
        body = """{}""".format(current_data_day_4[0])
        selection_day_4 = Selector(text=body).xpath("//div").extract()
        temp_day_4 = re.findall("\d+", selection_day_4[5])[0]

        current_data_day_5 = soup.find_all("div", attrs={"class": "day_5"})
        body = """{}""".format(current_data_day_5[0])
        selection_day_5 = Selector(text=body).xpath("//div").extract()
        temp_day_5 = re.findall("\d+", selection_day_5[5])[0]

        weather_network_data_now = (
            "Weather Network",
            current_day_temp,
            temp_day_1,
            temp_day_2,
            temp_day_3,
            temp_day_4,
            temp_day_5,
        )
    else:
        weather_network_data_now = (
            "Weather Network",
            "NA",
            "NA",
            "NA",
            "NA",
            "NA",
            "NA",
        )
except:
        weather_network_data_now = (
            "Weather Network",
            "NA",
            "NA",
            "NA",
            "NA",
            "NA",
            "NA",
        )
    
#url = req.json()[0]['url']
#code = req.json()[0]['code']

#ddict = defaultdict(list)

#for i in data:
#    if ddict[i["countrycode"]]:
#        ddict[i["countrycode"]].append(i)
#    else:
#        ddict[i["countrycode"]] = []
#        ddict[i["countrycode"]].append(i)

#info_urls = []        
#for k in ddict["IN"]:
#    if k["name"] == search_text.title():
#        info_urls.append(k['code'])
        
#print info_urls        
#try:
#    if info_urls:
#        data = "https://www.theweathernetwork.com/in/api/savedlocation/index/?placecodes={}".format(info_urls[0])
#        print data
#        url_req = requests.get(data)
#        print url_req.json()
#        temperature = url_req.json()[0]['temperature']
#        feels_like = url_req.json()[0]['feels_like']
#        updated = url_req.json()[0]['updatetime']
#        weather_network_data_now = ("Weather Network", str(temperature), "", "", "", "", "")
#    else:
#        weather_network_data_now = (
#            "Weather Network",
#            "Not Found",
#            "Not Found",
#            "Not Found",
#            "Not Found",
#            "Not Found",
#            "Not Found",
#        )
#except:
#    weather_network_data_now = (
#        "Weather Network",
#        "Not Found",
#        "Not Found",
#        "Not Found",
#        "Not Found",
#        "Not Foundr",
#        "Not Found",
#    )
#print weather_network_data_now

<h4> MSN Weather </h4>

In [5]:
try:
    base_url = "http://weather.service.msn.com/data.aspx?weasearchstr={}&culture=en-US&weadegreetype=F&src=outlook".format(search_text)
    req = requests.get(base_url)
    root = ET.fromstring(req.content)
    data = root.__dict__['_children'][0].__dict__['_children'][0].__dict__['attrib']
    data_1 = root.__dict__['_children'][0].__dict__["_children"][3].__dict__['attrib']['high']
    data_2 = root.__dict__['_children'][0].__dict__["_children"][4].__dict__['attrib']['high']
    data_3 = root.__dict__['_children'][0].__dict__["_children"][5].__dict__['attrib']['high']
    msn_weather_now = (
        "MSN Weather",
        int((int(data['temperature'])-32)*float(0.5556)),
        int((int(data_1)-32)*float(0.5556)),
        int((int(data_2)-32)*float(0.5556)),
        int((int(data_3)-32)*float(0.5556)),
        "NA",
        "NA",
    )
except Exception as e:
    msn_weather_now = (
        "MSN Weather",
        "NA",
        "NA",
        "NA",
        "NA",
        "NA",
        "NA",
    )
print msn_weather_now    

('MSN Weather', 32, 31, 31, 32, 'NA', 'NA')


<h4> Accurate Weather </h4>

In [6]:
search_url = "https://www.accuweather.com/en/search-locations"
search_data = {"s": search_text.title()}
try:
    req = requests.post(search_url, search_data)
    resp = req.__dict__['_content']

    soup = BeautifulSoup(resp, "lxml")
    data_input_box = soup.find_all("input", attrs={'id': 's'})

    data_str = str(data_input_box[0])
    data_url = re.findall("(https://www.accuweather.com/\w+/\w+/\w+/[0-9]+/[\w-]+/[0-9]+)\"", data_str)[0]

    base_url = data_url.replace("weather-forecast", "current-weather")
    base_link_url = base_url.replace("current-weather", "daily-weather-forecast")
    base_links = ["{}?day={}".format(base_link_url, i) for i in range(1, 6)]

    req = urllib2.Request(base_url)
    read_url = urllib2.urlopen(req)
    response = read_url.read()
    soup = BeautifulSoup(response, "lxml")
    five_day_data = soup.find_all("li", attrs={'data-href': base_links})

    column_header = ('Day', 'Date', 'real_feel', 'precipitation', 'description')
    data = []
    for day in five_day_data:
        hxs = HtmlXPathSelector(day).xpath("/html/body/p/text()").extract()
        doc = """{}""".format(hxs[0].encode("utf-8"))
        each_day = re.sub("\n+",
                          ",",
                          doc.decode("unicode_escape").encode("ascii", "ignore").replace("/", "").replace("More", "")
                         )
        data.append(filter(None, tuple(each_day.split(",")))) 
    accurate_weather_now = ("Accurate Weather", data[0][2], data[1][2], data[3][2], data[4][2], "NA", "NA")
except:
    accurate_weather_now = (
        "Accurate Weather",
        "NA",
        "NA",
        "NA",
        "NA",
        "NA",
        "NA",
    )
print accurate_weather_now

('Accurate Weather', '33', '33', '34', '35', 'NA', 'NA')


/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:26: ScrapyDeprecationWarning: scrapy.selector.HtmlXPathSelector is deprecated, instantiate scrapy.Selector instead.
/usr/local/lib/python2.7/dist-packages/parsel/selector.py:234: ScrapyDeprecationWarning: scrapy.selector.HtmlXPathSelector is deprecated, instantiate scrapy.Selector instead.
  for x in result]


<h4>Weather Bug Weather</h4>

In [7]:
# key = "a723563832c75742"
# pip install pywu
# base_url = "https://www.wunderground.com/forecast/in/{}".format(search_text)
base_url = "https://weather.weatherbug.com/api/localities/search?query={}".format(search_text)
slug_data = requests.get(base_url).json()[0]['SlugName']
data_url = "https://weather.weatherbug.com/weather-forecast/10-day-weather/{}".format(slug_data)

data_req = urllib2.urlopen(data_url)
response = data_req.read()
soup = BeautifulSoup(response, "lxml")
day_data = soup.find_all("div", attrs={"class": "day-card__desktop"})
temp_value = []
for each in day_data:
    body = """{}""".format(each)
    body = body.decode("utf-8").encode('ascii','ignore').replace("\n", "").replace("\r", "")
    select = Selector(text=body).xpath('//div[@class="temp"]/text()').extract()
    temp = int(round((int(select[0].strip()) - 32) * float(0.5556)))
    temp_value.append(temp)
weather_bug_data = ("Weather Bug", temp_value[0], temp_value[1], temp_value[2], temp_value[3], temp_value[4], temp_value[5])

<h4>BBC Weather</h4>

In [41]:
search_url = "http://www.bbc.com/locator/default/en-GB/autocomplete.json?search={}&filter=international".format(search_text)

city_data = requests.get(search_url).json()
try:
    weather_data = []
    if city_data:
        city_data = city_data[0]
        city_id = city_data['id']
    
        weather_url = "http://www.bbc.com/weather/{}".format(city_id)
        data_req = urllib2.urlopen(weather_url)
        response = data_req.read()
        soup = BeautifulSoup(response, "lxml")
        day_data = soup.find_all("td", attrs={"class": "min-temp"})
        for each in day_data:
            body = """{}""".format(each)
            body = body.decode("utf-8").encode('ascii','ignore')
            select = Selector(text=body).xpath('//span[@data-unit="c"]/text()').extract()
            weather_data.append(int(select[0]))
        bbc_weather_data = ("BBC Weather", weather_data[0], weather_data[1], weather_data[2], weather_data[3], weather_data[4], "NA")
    else:
        bbc_weather_data = ("BBC Weather", "NA", "NA", "NA", "NA", "NA", "NA")
except Exception as e:
    print e.message
    bbc_weather_data = ("BBC Weather", "NA", "NA", "NA", "NA", "NA", "NA")
print bbc_weather_data

('BBC Weather', 21, 21, 20, 20, 20, 'NA')


<h4>The Weather Channel</h4>

In [51]:
search_text = "bangalore"
search_url = "https://dsx.weather.com/x/v2/web/loc/en_US/1/4/5/9/11/13/19/21/1000/1001/1003/us^/{}?format=json&pg=0%2C10".format(search_text)
chennai = "https://dsx.weather.com/x/v2/web/loc/en_IN/1/4/5/9/11/13/19/21/1000/1001/1003/in^/chennai?api=7bb1c920-7027-4289-9c96-ae5e263980bc&format=json&pg=0%2C10"
bangalore = "https://dsx.weather.com/x/v2/web/loc/en_IN/1/4/5/9/11/13/19/21/1000/1001/1003/in^/bangalore?api=7bb1c920-7027-4289-9c96-ae5e263980bc&format=json&pg=0%2C10"
madurai = "https://dsx.weather.com/x/v2/web/loc/en_IN/1/4/5/9/11/13/19/21/1000/1001/1003/in^/madurai?api=7bb1c920-7027-4289-9c96-ae5e263980bc&format=json&pg=0%2C10"

<h2>Consolidated Weather Data</h2>

In [40]:
header = ("Sites", "Current Weather", "Day_1", "Day_2", "Day_3", "Day_4", "Day_5")
dataset = [
    yahoo_data_now,
    msn_weather_now,
    weather_network_data_now,
    accurate_weather_now,
    weather_bug_data,
    bbc_weather_data,
]
df = pd.DataFrame(data=dataset, columns=header)
print search_text.title()+"\n"
print df.set_index("Sites")

Bangalore

                 Current Weather Day_1 Day_2 Day_3 Day_4 Day_5
Sites                                                         
Yahoo                         29    28    29    30    31    31
MSN Weather                   32    31    31    32    NA    NA
Weather Network               33    30    31    31    32    31
Accurate Weather              33    33    34    35    NA    NA
Weather Bug                   30    30    31    31    32    32
BBC Weather                   21    21    20    20    20    NA
